XG Boost
=======

In [2]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.style.use("ggplot")

import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_fscore_support, auc, accuracy_score

In [3]:
data = pd.read_csv("../data/loans_sample_processed.csv", sep = "^")

In [4]:
data.head()

,funded_amnt_inv,installment,int_rate,annual_inc,dti,total_rec_late_fee,total_acc,manager,refinance,term_ 36 months,...,emp_length_< 1 year,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,application_type_Individual,application_type_Joint App,non_payment
0,25000.0,777.55,7.49,106157.0,9.37,0.00,49.0,0,1,1,...,0,0,1,0,0,0,0,1,0,0
1,12000.0,404.27,12.99,67000.0,21.33,0.00,28.0,1,1,1,...,0,0,1,0,0,0,0,1,0,1
2,4800.0,175.59,18.85,56000.0,7.62,0.00,15.0,0,1,1,...,0,0,0,0,0,0,1,1,0,0
3,7900.0,273.82,14.99,58300.0,12.15,0.00,17.0,0,1,1,...,0,0,1,0,0,0,0,1,0,0
4,10725.0,385.43,17.57,36000.0,21.68,19.27,18.0,0,1,1,...,0,0,0,0,0,0,1,1,0,0


In [5]:
data.shape

(49619, 38)

__Train / Test split:__

In [6]:
X = data.loc[:, data.columns!='non_payment']

In [7]:
y = data['non_payment']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=4290)

__Prior__:

In [9]:
y.value_counts()[0] / len(y)

0.7968923194743949

__XG Boost__

In [13]:
XG_train = xgb.DMatrix(X_train.values, label=y_train)

Grid parameters:

In [15]:
xg_param_grid = {'min_child_weight': (1, 20),
                 'colsample_bytree': (0.1, 1),
                 'max_depth': (5, 15),
                 'subsample': (0.5, 1),
                 'gamma': (0, 10),
                 'alpha': (0, 10)}

Cross validate model with Kfold stratified cross val

In [ ]:
xgb.cv(xg_param_grid, XG_train, num_boost_round=num_rounds, nfold=3, 
       seed=random_state, callbacks=[xgb.callback.early_stop(50)])

In [16]:
kfold = StratifiedKFold(n_splits=3)

In [92]:
gsRFC = GridSearchCV(random_forest, param_grid = rf_param_grid, cv = kfold, scoring = "roc_auc", n_jobs= 4, verbose = 1)

In [ ]:
gsRFC.fit(X_train,y_train)

Fitting 3 folds for each of 125 candidates, totalling 375 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.3min
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed: 13.4min
